In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/gutenberg-books-data/1661-0.txt


In [3]:
#! pip install nltk

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim

import numpy as np

from collections import   Counter

from torch.utils.data import Dataset, DataLoader

from nltk.tokenize import word_tokenize
import nltk





In [5]:
path = '/kaggle/input/gutenberg-books-data/1661-0.txt'
document = open(path).read()
print('corpus length:', len(document))

corpus length: 581888


In [6]:
document += """ You’ve cleaned your data, engineered brilliant features, and trained a model. You’ve validated and optimized it with an impressive F1-score. You are ready to push to production.

This isn’t about uploading a Jupyter notebook — it’s about orchestrating a controlled, monitored transition that brings your model to millions of real users without breaking anything in the process.

In the world of Production-Grade Systems, “deployment” is not a binary switch. It is a spectrum of risk and reward. As a Senior Engineer, you must choose the strategy that best balances safety with speed.

Deployment Strategies
1. Shadow Deployment: Validate Without Impact
The Concept
In a shadow deployment (also known as “dark launch”), production traffic is duplicated.

The current model responds to the user, while the new model processes the same data in the background without its output ever reaching the user.

Both models see the same traffic, but only the old model’s predictions are served to users.

The new model’s outputs are logged and compared offline.

How it works
Version 1 (current) → receives traffic → serves predictions to users ✅

Version 2 (new) → receives same traffic → logs predictions silently 📝

Compare both offline: latency, output distributions, edge case behaviour etc.

When to use it
In high-stakes domains like healthcare, finance, autonomous vehicles etc.

When deploying a fairly new model architecture that you want to validate exhaustively.

When you want zero real-world impact during validation

Pros
It has zero risk to users as the new version is solely deployed for observation and monitoring purpose.

It validates the model’s performance on real production traffic.

You can catch issues and edge cases before users experience them.

Cons
It doubles the computation cost as you will be running two models simultaneously.

You are not able to monitor real-world feedback on latency perceived by users.

It takes time to gather enough data for statistical confidence; hence you need to run both the versions (old + new) for long time.

2. Canary Deployment: Gradual, Controlled Rollout
The Concept
Route a small percentage of traffic (e.g., 5%) to the new model.

Monitor it closely and gradually increase traffic if metrics remain stable.

If metrics (latency, error rate, conversion) look good, you gradually increase the percentage (10% -> 50% -> 100%).

Rollout stages
Stage 1: v1 (95%) | v2 (5%) — monitor v2 closely with fine-grained alerts

Stage 2: v1 (80%) | v2 (20%) — still monitoring and comparing performance metrics

Stage 3: v1 (50%) | v2 (50%) — evaluating overall impact

Stage 4: v1 (0%) | v2 (100%) — full rollout completed

When to use it
In high-traffic applications where you want gradual user exposure.

If Models are too complex and have a hard-to-predict behaviours in production.

When catching issues early is critical but some exposure is acceptable

Pros
Mitigates risk by limiting initial exposure thereby reducing the blast radius if something goes wrong.

It provides real-time feedback on latency, load handling and cache behaviour.

It allows for fast rollback if issues emerge.

It enables you to detect long-tail issues (rare but impactful edge cases)

Cons
Slower to reach full deployment

Requires sophisticated traffic routing and monitoring

Stateful user experiences complicate rollout i.e. users seeing different models

3. Blue-Green Deployment: Instant Switchover
The Concept
Blue-Green deployment is a high-availability strategy designed to minimize downtime and reduce the risk of a release by maintaining two identical production environments.

The core of this strategy lies in the use of a Router or Load Balancer that can instantly redirect traffic between two environments, traditionally labelled “Blue” and “Green”.

Rollout stages
The Blue Environment: This is the “Current” live environment. It is currently serving 100% of production traffic using the existing model version.

The Green Environment: This is the “New” environment. It is an exact clone of the Blue environment’s infrastructure but hosts the new model version.

While Green is “idle” (not receiving public traffic), the engineering team performs final sanity checks and validation test in this live-cloned setting.

The Cutover: Once the Green environment is validated, the router switches all incoming traffic from Blue to Green instantly.

When to use it
Use it when the system requires near-zero downtime during updates.It is ideal for high-stakes environments where you need the ability to revert to a stable state immediately if the new model fails.This strategy should be used when the organization can afford to double its infrastructure costs.Pros Instantaneous Rollout: The transition happens at the routing layer, meaning users experience no downtime during the update.

Zero-Risk Rollback: If the new model (Green) begins to fail or produce unexpected results, you can switch the router back to Blue instantly, returning users to the stable, known state.

Isolated Testing: Because Green is identical to production but isolated from users, you can catch infrastructure-level bugs (like memory leaks or dependency mismatches) before a single user is affected.

Cons
Resource Intensive: This is the most expensive rollout strategy because it requires doubling your infrastructure.

You must pay for the compute and memory of two full production clusters simultaneously.

State Management: If your ML system relies on a database or a feature store that changes as users interact with the model, keeping the Blue and Green environments perfectly synchronized during the transition can be technically complex.

4. A/B Testing Deployment: Causal Proof of Impact
The Concept
This strategy is used by routing traffic randomly to two (or more) model versions.

You can measure the impact on key business metrics — not just technical metrics, but outcomes that matter: click-through rate, conversion rate, user engagement, revenue etc.



"""

In [7]:
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /usr/share/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

# Extracting Tokens and Generating Data

In [8]:
# tokenise

tokens = word_tokenize(document.lower())

In [9]:
# build vocab

vocab = {"<UNK>": 0}

for token in Counter(tokens).keys():
    #print(token)
    if token not in vocab:
        vocab[token] = len(vocab)

In [10]:
# extracting sentence from data

input_sentences = document.split("\n")

In [11]:
def text_to_indices(sentence, vocab):
    numerical_sentence = []

    for word in sentence:
        if word in vocab:
            numerical_sentence.append(vocab[word])
        else:
            numerical_sentence.append(vocab['<UNK>'])

    return numerical_sentence

In [12]:
# converting sentences to numerical tokens
input_numerical_sentences = []
for sentence in input_sentences:
    if sentence:
        input_numerical_sentences.append(text_to_indices(word_tokenize(sentence.lower()), vocab))

In [13]:
# generating padded training seq
training_seq = []
max_len = 0
padded_training_seq = []

for sentence in input_numerical_sentences:

    for i in range(1, len(sentence)):
        seq = sentence[:i+1]
        training_seq.append(sentence[:i+1])
        if len(seq)>max_len:
            max_len = len(seq)



for seq in training_seq:
    seq = [0]*(max_len-len(seq)) + seq
    #print(seq)
    padded_training_seq.append(seq)

padded_training_seq = torch.tensor(padded_training_seq, dtype=torch.long)

In [14]:
# generating X & y
X = padded_training_seq[:, :-1]
y = padded_training_seq[:, -1]

# Generating Dataset & DataLoader

In [15]:
# generating custom dataset

class CustomDataset(Dataset):

    def __init__(self, X, y):

        self.X = X
        self.y = y

    def __len__(self):
        return self.X.shape[0]

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [16]:
dataset = CustomDataset(X, y)

In [17]:
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [18]:
# for inp, out in dataloader:
#     print(inp, out)

# Model Building

In [19]:
class LSTMModel(nn.Module):

    def __init__(self, vocab_size, embedding_dimension, no_of_neurons):

        super().__init__()
        self.vocab_size = vocab_size
        self.embedding_dimension = embedding_dimension
        self.embedding = nn.Embedding(vocab_size, embedding_dimension)
        self.lstm = nn.LSTM(embedding_dimension, no_of_neurons, batch_first=True)
        self.output = nn.Linear(no_of_neurons, vocab_size)

    def forward(self, ques):
        embedded_ques = self.embedding(ques)
        intermeddiate_hidden_state, (final_hidden_state, final_cell_state) = self.lstm(embedded_ques)
        output = self.output(final_hidden_state.squeeze(0))
        return output
        
        
        

In [20]:
embedding_dimension, no_of_neurons = 100, 150
model = LSTMModel(len(vocab), embedding_dimension, no_of_neurons)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print(f"device: {device}")

learning_rate = 0.0001
epochs = 50

loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

device: cuda


In [21]:

total_loss = []

for epoch in range(epochs):
    
    for batch_x, batch_y in dataloader:
        batch_x, batch_y = batch_x.to(device), batch_y.to(device)

        optimizer.zero_grad()

        y_pred = model(batch_x)
        loss = loss_function(y_pred, batch_y)

        loss.backward()
        optimizer.step()

        total_loss.append(loss.item())

    print(f"for epoch: {epoch}/{epochs}, avg loss is {sum(total_loss)/len(total_loss)}")
        

for epoch: 0/50, avg loss is 6.192543764793778
for epoch: 1/50, avg loss is 5.907303421069575
for epoch: 2/50, avg loss is 5.732721591394201
for epoch: 3/50, avg loss is 5.601506417455019
for epoch: 4/50, avg loss is 5.494664381839984
for epoch: 5/50, avg loss is 5.403762524402655
for epoch: 6/50, avg loss is 5.324204175291626
for epoch: 7/50, avg loss is 5.253056709920511
for epoch: 8/50, avg loss is 5.188480683849271
for epoch: 9/50, avg loss is 5.129227634954578
for epoch: 10/50, avg loss is 5.074324729031412
for epoch: 11/50, avg loss is 5.023019718887938
for epoch: 12/50, avg loss is 4.974746580658129
for epoch: 13/50, avg loss is 4.929086029426224
for epoch: 14/50, avg loss is 4.885650362574226
for epoch: 15/50, avg loss is 4.844159717185516
for epoch: 16/50, avg loss is 4.80437989266748
for epoch: 17/50, avg loss is 4.766135772624976
for epoch: 18/50, avg loss is 4.7292360796396675
for epoch: 19/50, avg loss is 4.693589379321931
for epoch: 20/50, avg loss is 4.659053067149102
fo

In [22]:
max_len

66

In [30]:
def predict(model, vocab, query):

    tokenised = word_tokenize(query.lower())
    
    numerical_ques = text_to_indices(tokenised, vocab)
    
    padded_query = torch.tensor([0]*(max_len - len(numerical_ques)) + numerical_ques, dtype=torch.long).unsqueeze(0).to(device)
   
    next_word = model(padded_query)

    # print(next_word)
    # print(next_word.shape)

    value, index = torch.max(next_word, dim=1)
    # print(value)
    # print(index)

    str_next_word = list(vocab.keys())[index]
    # print(str_next_word)

    
    return query + " " + str_next_word


def next_n_words(n, model, vocab, query):
    
    for i in range(n):
        query = predict(model, vocab, query)
        print(query)

    return query


final_output = next_n_words(5, model, vocab, """“You had my note?” he""")

print(f"final_output: {final_output}")

“You had my note?” he asked
“You had my note?” he asked ,
“You had my note?” he asked , “
“You had my note?” he asked , “ and
“You had my note?” he asked , “ and i
final_output: “You had my note?” he asked , “ and i
